In [1]:
import IpynbImporter
import model as model
import view as view

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from view.ipynb


In [2]:
from PyQt5 import QtWidgets
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEnginePage
import sys
import os
import re
import time
import pymysql
import datetime
from apscheduler.triggers.cron import CronTrigger
# apscheduler 裡的 qt.py  第 37 行加這個   wait_seconds = int(wait_seconds) ，wait_seconds：當前距離下次任務啟動的時間差 (秒數)
from apscheduler.schedulers.qt import QtScheduler
from apscheduler.jobstores.base import ConflictingIdError
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import os 
# file directory
# 資料檔案
path = os.getcwd()
data_file_path = 'data_file'
data_file_path = os.path.join(path, data_file_path)
if not os.path.exists(data_file_path):
    os.mkdir(data_file_path)
# 設定檔案
setting_file_path = 'setting_file'
setting_file_path = os.path.join(path, setting_file_path)
if not os.path.exists(setting_file_path):
    os.mkdir(setting_file_path)

In [4]:
class AttributeError(BaseException, ):
    def __init__(self, error_str):
        self.text = error_str

In [5]:
class controller(view.Ui_GUI_crawler):
    def __init__(self,):
        self.is_success_connect_db=False
        self.db_dual_store=False
        self.db_datetime_store='no'
        self.schedule_crawler_list=[]
        self.app = QtWidgets.QApplication(sys.argv)
        self.MainWindow = QtWidgets.QMainWindow()
        self.setupUi(self.MainWindow)
        self.connect()
        file_list_url = self.get_file_directory("url")
        if file_list_url != []:
            self.url_comboBox.addItems(file_list_url)
        file_list_tag = self.get_file_directory("tag")
        if file_list_tag != []:
            self.tag_comboBox.addItems(file_list_tag)
        file_list_schedule = self.get_file_directory("schedule")
        if file_list_schedule != []:
            self.schedule_load_setting_comboBox.addItems(file_list_schedule)
        
    def connect(self):
        self.before_show_data.toggled.connect(lambda: self.change_show("before"))
        self.before_test_data.clicked.connect(lambda: self.test_data("before"))
        self.before_preview.clicked.connect(self.start_before)
        self.before_unknown.toggled.connect(self.allow_input)
        
        self.url_test_data.clicked.connect(lambda: self.test_data("url"))
        self.url_preview.clicked.connect(self.start_url_preview)
        self.url_exec.clicked.connect(self.start_url_exec)
        self.url_save_setting.clicked.connect(lambda: self.save_setting("url"))
        self.url_load_setting.clicked.connect(lambda: self.load_setting("url"))
        self.url_show_data.toggled.connect(lambda: self.change_show("url"))
        self.url_db_setting.clicked.connect(lambda: self.check_column("url"))
        
        self.tag_test_data.clicked.connect(lambda: self.test_data("tag"))
        self.tag_preview.clicked.connect(self.start_tag_preview)
        self.tag_next_page.clicked.connect(self.go_next_page)
        self.tag_exec.clicked.connect(self.start_tag_exec)
        self.tag_save_setting.clicked.connect(lambda: self.save_setting("tag"))
        self.tag_load_setting.clicked.connect(lambda: self.load_setting("tag"))
        self.tag_show_data.toggled.connect(lambda: self.change_show("tag"))
        self.tag_db_setting.clicked.connect(lambda: self.check_column("tag"))
        
        self.db_test_connect.clicked.connect(self.test_connect)
        self.db_datetime.clicked.connect(self.db_datetime_show)
        self.db_dual.clicked.connect(self.insert_cover)
        self.db_day.toggled.connect(self.datetime_status)
        
        self.schedule_day.toggled.connect(self.schedule_week_button_setting)
        self.schedule_create.clicked.connect(self.creat_job)
        self.schedule_show.clicked.connect(self.show_job)
        self.schedule_delete.clicked.connect(self.delete_job)
        self.schedule_exec_all.clicked.connect(self.exec_job)
        self.schedule_show_search.clicked.connect(self.show_job_partial)
        self.schedule_delete_search.clicked.connect(self.delete_job_partial)
        self.schedule_exec_all_search.clicked.connect(self.exec_job_partial)
    def start_before(self):
        self.before_preview.setEnabled(False)
        self.before_msg.clear()
        if self.before_url.text().strip() =="":
            self.printf("發送網址 不可為空！！", "before")
            self.before_preview.setEnabled(True)
            return
        
        url_attr = {'former_part':self.before_url.text().strip(), 'initial_pages':'', 'latter_part':'', 'end_pages':''}
        
        if self.before_domain.text().strip() == '(選填)' or self.before_domain.text().strip() == '':
            try:
                domain = re.search(r'//(.*?)/',url_attr['former_part']).group(1)
                self.before_domain.setText(domain)
            except:
                self.printf("網域名稱解析錯誤，請檢查網址的前半部分，或自行輸入網域！！", "before")
                self.before_preview.setEnabled(True)
                return
        else:
            domain = self.before_domain.text().strip()
            
        if self.before_urlencoded.isChecked():
            Content_Type = 'application/x-www-form-urlencoded'
        elif self.before_json.isChecked():
            Content_Type = 'application/json'
        elif self.before_unknown.isChecked():
            Content_Type = self.before_unknow_input.text().strip()
        header_attr = {'Host':domain,'Content-Type':Content_Type}
        
        if self.before_data.text().strip() != "":
            try:
                data = eval(self.before_data.text().strip())
                if type(data) != dict:
                    self.printf("標籤細項 型別不符，請以字典型式 {"":"","":""} 傳入各個 key-value！！", "before")
                    self.before_preview.setEnabled(True)
                    return
            except:
                self.printf("標籤細項 型別不符，請以字典型式 {"":"","":""} 傳入各個 key-value！！", "before")
                self.before_preview.setEnabled(True)
                return
        
        if self.before_GET.isChecked():
            method = "GET"
        elif self.before_POST.isChecked():
            method = "POST"
        
        if "my_test_crawler" not in str(dir(self)):
            self.my_test_crawler = model.site_url_Spider()
        self.my_test_crawler.update_attr('Untitled', header_attr, url_attr, '', '', '', data, method)
        self.printf("參數成功接收、實例化爬蟲完成，等待爬取...", "before") 
        try:
            self.my_test_crawler.before_craw()
        except Exception as ex:
            if 'Cloudflare' in str(ex):
                wording = 'Cloudflare 擋住了請求！'
            self.printf("爬取失敗：" + wording, "before")
            self.before_preview.setEnabled(True)
            return
        self.printf("爬取完成", "before")
        if self.before_show_html.isChecked():
            self.printf("正在加載 html ...", "before") 
            self.before_html_browser.setHtml(self.my_test_crawler.html)
            self.printf("加載完畢", 'before') 
        elif self.before_show_data.isChecked():
            self.printf("等待將 cookies 資料顯示出來...", "before") 
            self.before_html_browser.setHtml("<div>" + self.my_test_crawler.cookies.replace('), Cookie(','),<br>Cookie(') + "<div>")
            self.printf("顯示完成", 'before') 
        self.before_preview.setEnabled(True)
    def allow_input(self):
        if self.before_unknown.isChecked():
            self.before_unknow_input.setEnabled(True)
        else:
            self.before_unknow_input.setEnabled(False)


    def get_data_site_url_Spider(self):
        former_part = self.url_former_part.text().strip()
        if former_part =='':
            raise AttributeError('前半網址 不可為空！！')
        try:
            initial_pages = int(self.url_initial_pages.text().strip())
        except:
            raise AttributeError('初始頁數 請輸入正整數！！')
        latter_part = self.url_latter_part.text().strip()

        if self.url_end_pages.text().strip() =='':
            end_pages=initial_pages
            self.url_end_pages.setText(str(initial_pages))
        else:
            try:
                end_pages = int(self.url_end_pages.text().strip())
            except Exception as ex:
                raise AttributeError('結束頁數 請輸入正整數！！')
        url_attr = {'former_part':former_part, 'initial_pages':initial_pages, 'latter_part':latter_part, 'end_pages':end_pages}
        if self.url_domain.text().strip() =='(選填)' or self.url_domain.text().strip() == '':
            try:
                domain = re.search(r'//(.*?)/',former_part).group(1)
                self.url_domain.setText(domain)
            except:
                raise AttributeError('網域名稱解析錯誤，請檢查網址的前半部分，或自行輸入網域！！')
        else:
            domain = self.url_domain.text().strip()
        if self.url_name.text().strip() == '' or self.url_name.text().strip() == '(選填)':
            self.name = 'Untitled'
            self.url_name.setText(self.name)
        else:
            self.name = self.url_name.text().strip()
        
        header_attr = {'Host':domain,'Content-Type':None}
        tag_section = self.url_tag_section.text().strip()

        if self.url_tag_detail.toPlainText().strip() =="":
            raise AttributeError('標籤細項 不可為空！！')
        try:
            tag_detail = eval(self.url_tag_detail.toPlainText().strip())
        except:
            raise AttributeError('標籤細項 拆分失敗，請以括號()分隔不同標籤！！')
        if not(type(tag_detail) != list or type(tag_detail) != tuple):
            raise AttributeError('標籤細項 型別不符，請以括號()分隔不同標籤！！')
        if "my_test_crawler" not in str(dir(self)):
            self.my_test_crawler = model.site_url_Spider()
        self.my_test_crawler.update_attr(self.name, header_attr, url_attr, '', tag_section, tag_detail, None, None)
        self.printf("參數成功接收，等待爬取...", "url")  
    def start_url_preview(self):
        self.url_preview.setEnabled(False)
        self.url_msg.clear()
        try:
            self.get_data_site_url_Spider()
        except AttributeError as ex:
            self.url_preview.setEnabled(True)
            self.printf(str(ex), "url")
            return
        
        self.my_test_crawler.test_parse(data_export = True, next_url = False)
        self.printf("測試爬取完成，如資料錯誤，請檢查欲爬取資料是否存在 html 裡", "url")
        
        if self.url_show_html.isChecked():
            self.printf("等待將 html 頁面顯示出來...", "url") 
            self.url_html_browser.setHtml(self.my_test_crawler.html)
            self.printf("加載完畢",'url') 
        elif self.url_show_data.isChecked():
            self.printf("等待將 json 資料顯示出來...", "url") 
            self.url_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
            self.printf(f"顯示完成，總筆數：{len(self.my_test_crawler.json)}","url") 
        self.download_file("url")
        
        self.url_exec.setEnabled(True)
        self.url_preview.setEnabled(True)
        self.url_save_setting.setEnabled(True)
        
        self.url_no.setEnabled(True)
        self.url_excel.setEnabled(True)
        self.url_json.setEnabled(True)        
        if self.is_success_connect_db:
            self.url_db_setting.setEnabled(True)
    def start_url_exec(self,):
        self.url_exec.setEnabled(False)
        self.url_msg.clear()
        self.url_html_browser.setHtml('')
        try:
            self.get_data_site_url_Spider()
        except AttributeError as ex:
            self.url_exec.setEnabled(True)
            self.printf(str(ex), "url")
            return
        self.my_test_crawler.url_loop_parse()
        self.printf(self.my_test_crawler.str, "url")
        
        if self.url_show_html.isChecked():
            self.printf("等待將 html 頁面顯示出來...", "url") 
            self.url_html_browser.setHtml(self.my_test_crawler.html)
            self.printf("加載完畢",'url') 
        elif self.url_show_data.isChecked():
            self.printf("等待將 json 資料顯示出來...", "url") 
            self.url_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
            self.printf(f"顯示完成，總筆數：{len(re.findall(r'}, {',self.my_test_crawler.json))+1}","url") 
        self.download_file("url")
        self.url_exec.setEnabled(True)
        
    def get_data_next_tag_Spider(self):
        former_part = self.tag_former_part.text().strip()
        if former_part =="":
            raise AttributeError('網址 不可為空！！')
        url_attr = {'former_part':former_part, 'initial_pages':'', 'latter_part':'', 'end_pages':''}
        
        if self.tag_domain.text().strip() == '(選填)' or self.tag_domain.text().strip() == '':
            try:
                domain = re.search(r'//(.*?)/',former_part).group(1)
                self.tag_domain.setText(domain)
            except:
                raise AttributeError('網域名稱解析錯誤，請檢查網址的前半部分，或自行輸入網域！！')
        else:
            domain = self.tag_domain.text().strip()
            
        header_attr = {'Host':domain,'Content-Type':None}
        
        if self.tag_name.text().strip() == '' or self.tag_name.text().strip() == '(選填)':
            self.name = 'Untitled'
            self.tag_name.setText(self.name)
        else:
            self.name = self.tag_name.text().strip()
        
        try:
            limit = int(self.tag_upper_limit.text().strip())
        except:
            raise AttributeError('跳轉上限 請輸入正整數！！')

        
        
        if self.tag_next_tag.text().strip() =="":
            raise AttributeError('下一頁tag 不可為空！！')
        try:
            next_tag = eval(self.tag_next_tag.text().strip())+tuple([limit])
        except:
            raise AttributeError('下一頁tag 型別不符！！')
        tag_section = self.tag_tag_section.text().strip()
        if self.tag_tag_detail.toPlainText().strip() =="":
            raise AttributeError('標籤細項 不可為空！！')
        try:
            tag_detail = eval(self.tag_tag_detail.toPlainText().strip())
        except:
            raise AttributeError('標籤細項 拆分失敗，請以括號()分隔不同標籤！！')
        if not(type(tag_detail) != list or type(tag_detail) != tuple):
            raise AttributeError('標籤細項 型別不符，請以括號()分隔不同標籤')
        if "my_test_crawler" not in str(dir(self)):
            self.my_test_crawler = model.site_url_Spider()
        self.my_test_crawler.update_attr(self.name, header_attr, url_attr, next_tag, tag_section, tag_detail, None, None)
        self.printf("參數成功接收，等待爬取...", "tag")  
    def start_tag_preview(self):
        self.tag_preview.setEnabled(False)
        self.tag_msg.clear()
        try:
            self.get_data_next_tag_Spider()
        except AttributeError as ex:
            self.tag_preview.setEnabled(True)
            self.printf(str(ex), "tag")
            return
        
        self.my_test_crawler.test_parse(data_export = True, next_url = True)
        self.printf("測試爬取完成，如資料錯誤，請檢查欲爬取資料是否存在 html 裡", "tag")
        
        if self.tag_show_html.isChecked():
            self.printf("等待將 html 頁面顯示出來...", "tag") 
            self.tag_html_browser.setHtml(self.my_test_crawler.html)
            self.printf("加載完畢",'url') 
        elif self.tag_show_data.isChecked():
            self.printf("等待將 json 資料顯示出來...", "tag") 
            self.tag_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
            self.printf(f"顯示完成，總筆數：{len(self.my_test_crawler.json)}","tag") 
        self.download_file("tag")
        
        self.tag_next_url.setText(self.my_test_crawler.next_url)
        self.tag_exec.setEnabled(True)
        self.tag_preview.setEnabled(True)
        self.tag_save_setting.setEnabled(True)
        self.tag_next_page.setEnabled(True)
        
        self.tag_no.setEnabled(True)
        self.tag_excel.setEnabled(True)
        self.tag_json.setEnabled(True)        
        if self.is_success_connect_db:
            self.tag_db_setting.setEnabled(True)
    def go_next_page(self):
        self.my_test_crawler.url = self.my_test_crawler.next_url
        self.tag_next_url.setText('')
        self.tag_former_part.setText(self.my_test_crawler.url)
        self.start_tag_preview()
        
    def start_tag_exec(self,):
        self.tag_exec.setEnabled(False)
        self.tag_msg.clear()
        self.tag_html_browser.setHtml('')
        try:
            self.get_data_next_tag_Spider()
        except AttributeError as ex:
            self.tag_exec.setEnabled(True)
            self.printf(str(ex), "tag")
            return
        self.my_test_crawler.next_loop_parse()
        self.printf(self.my_test_crawler.str, "tag")
        
        if self.tag_show_html.isChecked():
            self.printf("等待將 html 頁面顯示出來...", "tag") 
            self.tag_html_browser.setHtml(self.my_test_crawler.html)
            self.printf("加載完畢",'url') 
        elif self.tag_show_data.isChecked():
            self.printf("等待將 json 資料顯示出來...", "tag") 
            self.tag_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
            self.printf(f"顯示完成，總筆數：{len(re.findall(r'}, {',self.my_test_crawler.json))+1}","tag") 
        self.download_file("tag")
        self.tag_exec.setEnabled(True)
    def get_file_directory(self, typee):
        file_path = setting_file_path
        files = os.listdir(file_path)
        my_file_list =[]
        if len(files) == 0:
            self.printf(f"當前 {file_path} 資料夾沒有可讀取檔案...",typee) 
            return []
        for i in files:
            if file_path == data_file_path and (re.search(r'.txt',i) or re.search(r'.excel',i)):
                my_file_list.append(i)
            elif file_path == setting_file_path and re.search(r'.ini',i):
                if typee =="url" and re.search(r'url',i):
                    my_file_list.append(i)
                elif typee =="tag" and re.search(r'tag',i):
                    my_file_list.append(i)
                elif typee =="schedule":
                    my_file_list.append(i)
        if len(my_file_list) == 0:
            self.printf(f"當前 {file_path} 資料夾內沒有可讀取檔案...",typee) 
            return []
        return my_file_list
        # xx = get_file_directory("url")

    def save_setting(self, typee):
        setting_file_name = self.name + "_setting_file"
        file_list = self.get_file_directory(typee)
        x = 0
        while True:
            x = x + 1
            self.file_name = typee + '_' + setting_file_name + '_' + str(x) + '.ini'
            if self.file_name not in file_list:
                break
        crawler_ini = {}
        crawler_ini['before_url'] = self.before_url.text().strip().replace('%','%%')
        crawler_ini['before_data'] = self.before_data.text().strip().replace('%','%%')
        crawler_ini['before_domain'] = self.before_domain.text().strip().replace('%','%%')
        if self.before_GET.isChecked():
            method = "GET"
        elif self.before_POST.isChecked():
            method = "POST"
        crawler_ini['before_method'] = method
        if self.before_urlencoded.isChecked():
            Content_Type = 'application/x-www-form-urlencoded'
        elif self.before_json.isChecked():
            Content_Type = 'application/json'
        elif self.before_unknown.isChecked():
            Content_Type = self.before_unknow_input.text().strip().replace('%','%%')
        crawler_ini['before_content_type'] = Content_Type
        if typee =="url":
            crawler_ini['former_part'] = self.url_former_part.text().strip().replace('%','%%')
            crawler_ini['initial_pages'] = self.url_initial_pages.text().strip().replace('%','%%')
            crawler_ini['latter_part'] = self.url_latter_part.text().strip().replace('%','%%')
            crawler_ini['end_pages'] = self.url_end_pages.text().strip().replace('%','%%')
            crawler_ini['next_url'] = ''
            crawler_ini['next_tag'] = ''
            crawler_ini['domain'] = self.url_domain.text().strip().replace('%','%%')
            crawler_ini['name'] = self.url_name.text().strip().replace('%','%%')
            crawler_ini['tag_section'] = self.url_tag_section.text().strip().replace('%','%%')
            crawler_ini['tag_detail'] = self.url_tag_detail.toPlainText().strip().replace('%','%%')
            crawler_ini['upper_limit'] = ''
            if self.url_excel.isChecked():
                data_download = "excel"
            elif self.url_json.isChecked():
                data_download = "json"
            elif self.url_no.isChecked():
                data_download = "no"
            elif self.url_db_setting.isChecked():
                data_download = "sql"
            crawler_ini['data_download'] = data_download
        elif typee =="tag":
            crawler_ini['former_part'] = self.tag_former_part.text().strip().replace('%','%%')
            crawler_ini['initial_pages'] = ''
            crawler_ini['latter_part'] = ''
            crawler_ini['end_pages'] = ''
            crawler_ini['next_url'] = self.tag_next_url.text().strip().replace('%','%%')
            crawler_ini['next_tag'] = self.tag_next_tag.text().strip().replace('%','%%')
            crawler_ini['domain'] = self.tag_domain.text().strip().replace('%','%%')
            crawler_ini['name'] = self.tag_name.text().strip().replace('%','%%')
            crawler_ini['tag_section'] = self.tag_tag_section.text().strip().replace('%','%%')
            crawler_ini['tag_detail'] = self.tag_tag_detail.toPlainText().strip().replace('%','%%')
            crawler_ini['upper_limit'] = self.tag_upper_limit.text().strip().replace('%','%%')
            if self.tag_excel.isChecked():
                data_download = "excel"
            elif self.tag_json.isChecked():
                data_download = "json"
            elif self.tag_no.isChecked():
                data_download = "no"
            elif self.tag_db_setting.isChecked():
                data_download = "sql"
            crawler_ini['data_download'] = data_download

        crawler_ini['host'] = self.db_ip.text().strip()
        crawler_ini['port'] = self.db_port.text().strip()
        crawler_ini['user'] = self.db_username.text().strip()
        crawler_ini['password'] = self.db_password.text().strip()
        crawler_ini['db'] = self.db_name.text().strip()
        crawler_ini['charset'] = self.db_charset.text().strip()
        crawler_ini['autocommit'] = True
        crawler_ini['table_name'] = self.db_table_name.text().strip()
        crawler_ini['is_datetime'] = self.db_datetime_store
        crawler_ini['is_cover'] = self.db_dual_store
        model.save_setting_ini(setting_file_path + '/' + self.file_name, crawler_ini)
        self.printf(f"檔案保存成功，檔名：{self.file_name}，檔名請勿修改",typee) 
    
    
        file_list_schedule = self.get_file_directory("schedule")
        if file_list_schedule != []:
            self.schedule_load_setting_comboBox.clear()
            self.schedule_load_setting_comboBox.addItems(file_list_schedule)
            
        file_list = self.get_file_directory(typee)
        if typee =="url":
            self.url_comboBox.clear()
            self.url_comboBox.addItems(file_list)
        elif typee =="tag":
            self.tag_comboBox.clear()
            self.tag_comboBox.addItems(file_list)
    def load_setting(self,typee):
        if typee =="url":
            comboBox = self.url_comboBox
        elif typee =="tag":
            comboBox = self.tag_comboBox
        this_file_name = comboBox.currentText().strip()
        path_name = setting_file_path + '/' + this_file_name
        
        try:
            data,data_before,db_settings,db_settings_others = model.load_setting_ini(path_name)
            self.printf(f"{this_file_name} 讀取成功",typee) 
        except Exception as ex:
            print(ex)
            self.printf(f"{this_file_name} 讀取失敗，請檢查檔案是否存在",typee) 
            return
        if data_before['before_url'] != '' and "選填" not in data_before['before_domain']:
            self.before_url.setText(data_before['before_url'])
            self.before_data.setText(data_before['before_data'])
            self.before_domain.setText(data_before['before_domain'])
            method =  data_before['before_method']
            if method == "GET":
                self.before_GET.setChecked(True)
            elif method == "POST":
                self.before_POST.setChecked(True)
            content_type = data_before['before_content_type']
            if content_type == 'application/x-www-form-urlencoded':
                self.before_urlencoded.setChecked(True)
            elif content_type == 'application/json':
                self.before_json.setChecked(True)
            else:
                self.before_unknown.setChecked(True)
                self.before_unknow_input.setText(content_type)
            try:
                self.start_before()
                if self.my_test_crawler.cookies == '[]':
                    self.printf(f"{this_file_name} 執行前置動作，未成功獲取 cookies",typee) 
            except Exception as ex:
                print(ex)
                self.printf(f"{this_file_name} 執行前置動作失敗，請重新測試",typee) 

        else:
            self.printf(f"{this_file_name} 未設定前置動作，略過",typee) 
            
        if typee =="url":
            self.url_former_part.setText(data['former_part'])
            self.url_initial_pages.setText(data['initial_pages'])
            self.url_latter_part.setText(data['latter_part'])
            self.url_end_pages.setText(data['end_pages'])
            self.url_domain.setText(data['domain'])
            self.url_name.setText(data['name'])
            self.url_tag_section.setText(data['tag_section'])
            self.url_tag_detail.setText(data['tag_detail'])
            try:
                self.get_data_site_url_Spider()
            except AttributeError as ex:
                self.url_preview.setEnabled(True)
                self.printf(str(ex), typee)
                return
        elif typee =="tag":
            self.tag_former_part.setText(data['former_part'])
            self.tag_next_tag.setText(data['next_tag'])
            self.tag_next_url.setText(data['next_url'])
            self.tag_domain.setText(data['domain'])
            self.tag_name.setText(data['name'])
            self.tag_tag_section.setText(data['tag_section'])
            self.tag_tag_detail.setText(data['tag_detail'])
            self.tag_upper_limit.setText(data['upper_limit'])
            try:
                self.get_data_next_tag_Spider()
            except AttributeError as ex:
                self.tag_preview.setEnabled(True)
                self.printf(str(ex), typee)
                return
    def change_show(self, typee):
        try: 
            if typee =="url":
                if self.url_show_html.isChecked():
                    self.printf("正在加載 html ...",typee) 
                    self.url_html_browser.setHtml(self.my_test_crawler.html)
                    self.printf("加載完畢",'url') 
                elif self.url_show_data.isChecked():
                    self.printf("等待將 json 資料顯示出來...",typee) 
                    self.url_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
                    self.printf(f"顯示完成，總筆數：{len(re.findall(r'}, {',self.my_test_crawler.json))+1}",typee) 
            elif typee =="before":
                if self.before_show_html.isChecked():
                    self.printf("正在加載 html ...",typee) 
                    self.before_html_browser.setHtml(self.my_test_crawler.html)
                    self.printf("加載完畢",'before') 
                elif self.before_show_data.isChecked():
                    self.printf("等待將 cookies 資料顯示出來...", typee) 
                    self.before_html_browser.setHtml("<div>" + self.my_test_crawler.cookies.replace('), Cookie(','),<br>Cookie(') + "<div>")
                    self.printf("顯示完成", typee) 
            elif typee =="tag":
                if self.tag_show_html.isChecked():
                    self.printf("正在加載 html ...",typee) 
                    self.tag_html_browser.setHtml(self.my_test_crawler.html)
                    self.printf("加載完畢",'url') 
                elif self.tag_show_data.isChecked():
                    self.printf("等待將 json 資料顯示出來...",typee) 
                    self.tag_html_browser.setHtml("<div>"+str(self.my_test_crawler.json)[2:-2].replace('}, {',',<br>')+"<div>")
                    self.printf(f"顯示完成，總筆數：{len(re.findall(r'}, {',self.my_test_crawler.json))+1}",typee) 
        except AttributeError:
            pass
        except Exception as ex:
            self.printf('爬蟲未正確執行，請確認執行是否成功',typee)

    def download_file(self, typee):
        try:
            result = time.strftime("%Y%m%d_%H%M%S", time.localtime())
            if typee =="url":
                if self.url_excel.isChecked():
                    self_file_path = data_file_path + '/' + self.name + '_' +  result + '.xlsx'
                    self.my_test_crawler.export_excel(self_file_path)
                    self.printf(f"檔案製作完成，檔案位置: {self_file_path}", typee)
                elif self.url_json.isChecked():
                    self_file_path = data_file_path  + '/' + self.name + '_' +  result + '.txt'
                    with open(self_file_path, 'a+', encoding='utf-8') as file:
                        file.write(str(self.my_test_crawler.json))
                        file.close()
                    self.printf(f"檔案製作完成，檔案位置: {self_file_path}", typee)
                elif self.url_db_setting.isChecked():
                    x=0
                    dics = json.loads(self.my_test_crawler.json)
                    conn = pymysql.connect(**self.db_settings)
                    print(self.db_datetime_store)
                    with conn.cursor() as cursor:
                        for data in dics:
                            if self.db_datetime_store=='sec':
                                data['Data_Datetime']=datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S")
                            elif self.db_datetime_store=='day':
                                data['Data_Datetime']=datetime.datetime.strftime(datetime.date.today(), "%Y-%m-%d")
                            command_first = f"INSERT INTO {self.db_settings['db']}.{self.table_name} ({', '.join(data.keys())}) VALUES "
                            command_second = '("'+ '\", \"'.join(data.values()) +'") '
                            if self.db_dual_store:
                                update_query ='on DUPLICATE KEY UPDATE ' +  u', '.join([u"{}='{}'".format(k, v) for k, v in zip(data.keys(),data.values())])
                                command = command_first+command_second + update_query
                            else:
                                command = command_first+command_second
                            try:
                                cursor.execute(command)
                            except Exception as ex:
                                self.printf(f"{data} 存入資料庫報錯，錯誤內容: {ex}", typee)
                                continue
                            x=x+1
                    self.printf(f"資料庫儲存完成，資料共有 {len(dics)} 筆，成功存入 {x} 筆", typee)
                else:
                    return
            elif typee =="tag":
                if self.tag_excel.isChecked():
                    self_file_path = data_file_path + '/' + self.name + '_' +  result + '.xlsx'
                    self.my_test_crawler.export_excel(self_file_path)
                    self.printf(f"檔案製作完成，檔案位置: {self_file_path}", typee)
                elif self.tag_json.isChecked():
                    self_file_path = data_file_path  + '/' + self.name + '_' +  result + '.txt'
                    with open(self_file_path, 'a+', encoding='utf-8') as file:
                        file.write(str(self.my_test_crawler.json))
                        file.close()
                    self.printf(f"檔案製作完成，檔案位置: {self_file_path}", typee)
                elif self.tag_db_setting.isChecked():
                    x=0
                    dics = json.loads(self.my_test_crawler.json)
                    conn = pymysql.connect(**self.db_settings)
                    with conn.cursor() as cursor:
                        for data in dics:
                            if self.db_datetime_store=='sec':
                                data['Data_Datetime']=datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S")
                            elif self.db_datetime_store=='day':
                                data['Data_Datetime']=datetime.datetime.strftime(datetime.date.today(), "%Y-%m-%d")
                            command_first = f"INSERT INTO {self.db_settings['db']}.{self.table_name} ({', '.join(data.keys())}) VALUES "
                            command_second = '("'+ '\", \"'.join(data.values()) +'") '
                            if self.db_dual_store:
                                update_query ='on DUPLICATE KEY UPDATE ' +  u', '.join([u"{}='{}'".format(k, v) for k, v in zip(data.keys(),data.values())])
                                command = command_first+command_second + update_query
                            else:
                                command = command_first+command_second
                            try:
                                cursor.execute(command)
                            except Exception as ex:
                                self.printf(f"{data} 存入資料庫報錯，錯誤內容: {ex}", typee)
                                continue
                            x=x+1
                    self.printf(f"資料庫儲存完成，資料共有 {len(dics)} 筆，成功存入 {x} 筆", typee)
                else:
                    return
            
        except AttributeError:
            pass
        except Exception as ex:
            self.printf(str(ex), typee)
    def test_connect(self):
        if self.db_datetime.isChecked():
            if not(self.db_day.isChecked() or self.db_sec.isChecked()):
                self.printf('最小時間單位未選擇', 'db')
                return
        #self.test_data('db')
        if self.db_ip.text().strip() =='' or self.db_port.text().strip() =='':
            self.printf('請填入伺服器IP和端口號', 'db')
            return
        if self.db_table_name.text().strip() =='' or self.db_name.text().strip() =='':
            self.printf('請填入資料庫名稱和資料表名稱', 'db')
            return 
        if self.db_username.text().strip() =='' or self.db_password.text().strip() =='':
            self.printf('請填入登入用帳號密碼', 'db')
            return 
        self.db_settings = {
        "host": self.db_ip.text().strip(),
        "port": int(self.db_port.text().strip()),
        "user": self.db_username.text().strip(),
        "password": self.db_password.text().strip(),
        "db": self.db_name.text().strip(),
        "charset": self.db_charset.text().strip(),
        "autocommit":True}
        self.table_name = self.db_table_name.text().strip()
        try:
            # 建立Connection物件
            conn = pymysql.connect(**self.db_settings)
            # 建立Cursor物件
            with conn.cursor(pymysql.cursors.DictCursor) as cursor:
              #資料表相關操作
                # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
                command = f"SHOW COLUMNS FROM {self.db_settings['db']}.{self.table_name}"
                # 執行
                cursor.execute(command)
                table_des = cursor.fetchall()
        except Exception as ex:
            self.printf('資料庫連線失敗' + str(ex), 'db')
            return
        self.column_name_null=[i['Field']  for i in table_des if i['Null']=='YES' or (i['Null']=='NO' and'auto_increment' in i['Extra'])]
        self.column_name_notnull=[i['Field']  for i in table_des if i['Null']=='NO' and 'auto_increment' not in i['Extra']]
        self.printf(f"資料庫連線成功！必填欄位名稱：{str(self.column_name_notnull)}；非必填欄位名稱：{str(self.column_name_null)}", 'db')
        if self.db_datetime_store!='no' and ('Data_Datetime' not in self.column_name_notnull and 'Data_Datetime' not in self.column_name_null):
            self.printf(f"資料庫欄位未出現 Data_Datetime，取消記錄日期時間", 'db')
            self.db_datetime.setChecked(False)
            self.db_datetime_show()
        elif self.db_datetime_store!='no':
             self.printf(f"資料庫欄位存在 Data_Datetime，記錄日期時間", 'db')
        if self.db_dual_store:
            self.printf(f"開啟自動更新功能", 'db')
        
        self.is_success_connect_db=True
        if self.url_exec.isEnabled():
            self.url_db_setting.setEnabled(True)
        if self.tag_exec.isEnabled():
            self.tag_db_setting.setEnabled(True)
        self.printf(' ', 'db')
    def check_column(self, typee):
        if typee =="url":
            if self.url_tag_detail.toPlainText().strip()=='':
                self.printf("標籤設定細項設定不可為空，否則無法判斷資料庫欄位", typee)
                self.url_no.setChecked(True)
                return
            diff_set = set([i[0] for i in eval(self.url_tag_detail.toPlainText().strip())]) - set(self.column_name_null+self.column_name_notnull)
            if len(diff_set) >0:
                self.printf(f"資料表欄位無法對齊：{str(diff_set)}非在資料表欄位中", typee)
                self.url_no.setChecked(True)
                return
            diff_set = set(self.column_name_notnull) - set([i[0] for i in eval(self.url_tag_detail.toPlainText().strip())])
            if len(diff_set) >0:
                self.printf(f"資料表必填欄位無法滿足：{str(diff_set)} 資料表欄位沒有賦值", typee)
                self.url_no.setChecked(True)
                return
        elif typee =="tag":
            if self.tag_tag_detail.toPlainText().strip()=='':
                self.printf("標籤設定細項設定不可為空，否則無法判斷資料庫欄位", typee)
                self.tag_no.setChecked(True)
                return
            diff_set = set([i[0] for i in eval(self.tag_tag_detail.toPlainText().strip())]) - set(self.column_name_null+self.column_name_notnull)
            if len(diff_set) >0:
                self.printf(f"資料表欄位無法對齊：{str(diff_set)}非在資料表欄位中", typee)
                self.tag_no.setChecked(True)
                return
            diff_set = set(self.column_name_notnull) - set([i[0] for i in eval(self.tag_tag_detail.toPlainText().strip())])
            if len(diff_set) >0:
                self.printf(f"資料表必填欄位無法滿足：{str(diff_set)} 資料表欄位沒有賦值", typee)
                self.tag_no.setChecked(True)
                return
        self.printf(f"資料表欄位檢查完成！", typee)
    def db_datetime_show(self):
        if self.db_datetime.isChecked():
            self.db_day.setEnabled(True)
            self.db_sec.setEnabled(True)
            if self.db_day.isChecked():
                self.db_datetime_store='day'
            elif self.db_sec.isChecked():
                self.db_datetime_store='sec'
        else:
            self.db_day.setChecked(False)
            self.db_day.setEnabled(False)
            self.db_sec.setChecked(False)
            self.db_sec.setEnabled(False)
            self.db_datetime_store='no'
    def datetime_status(self):
        if self.db_day.isChecked():
            self.db_datetime_store='day'
        elif self.db_sec.isChecked():
            self.db_datetime_store='sec'
    def insert_cover(self):
        if self.db_dual.isChecked():
            self.db_dual_store=True
        else:
            self.db_dual_store=False
    def test_data(self, typee):
        if typee == "url":
            self.url_former_part.setText('https://web.pcc.gov.tw/prkms/tender/common/basic/readTenderBasic?tenderEndDate=2022%2F06%2F01&orgName=&tenderName=&searchType=basic&d-49738-p=')
            self.url_initial_pages.setText('1')
            self.url_latter_part.setText('&firstSearch=false&pageSize=50&radProctrgCate=&tenderId=&orgId=&tenderStartDate=2022%2F06%2F01&tenderType=TENDER_DECLARATION&dateType=isDate&tenderWay=TENDER_WAY_ALL_DECLARATION&level_1=on')
            self.url_tag_section.setText('div#printArea table.tb_01 tr')
            self.url_tag_detail.setText("[('機關名稱','td',1,'text'),('標案案號','td',2,'text'),('連結','td a ',0,'href'),('傳輸次數','td',3,'text'),('招標方式','td',4,'text')]")
        elif typee == "before":
            self.before_url.setText("https://sso.teachable.com/secure/teachable_accounts/sign_in")
            xx={"utf8":"✓","authenticity_token":"PQYVq3q5auCLwa/Wi+UNdu3u4kMQqYSBN+u3k6Q5qpBH1O+egi+IohEBkQqH5LPa456xzGJfrddbLV9D++6srQ==","teachable_account[email]":"teciram904@saturdata.com","teachable_account[password]":"teciram904","commit":"Log In"}
            self.before_data.setText(str(xx))
        elif typee == "tag":
            self.tag_former_part.setText('https://www.ptt.cc/bbs/LoL/index.html')
            self.tag_next_url.setText('')
            self.tag_next_tag.setText('("div.btn-group.btn-group-paging a",1)')
            self.tag_upper_limit.setText('9')
            self.tag_tag_section.setText('div.r-ent')
            self.tag_tag_detail.setText("[('title','div.title',0,'text'),('author','div.author',0,'text'),('date','div.date',0,'text'),('url','div.title a',0,'href')]")
        elif typee =="db":
            self.db_ip.setText('127.0.0.1')
            self.db_port.setText('3306')
            self.db_username.setText('root')
            self.db_password.setText('As123459362')
            self.db_name.setText('pttcrawler')
            self.db_charset.setText('utf8mb4')
            self.db_table_name.setText('ptt_data')
        self.schedule_name.setText('test')
        xx = datetime.datetime.now() + datetime.timedelta(minutes=1)
        date_time = xx.strftime("%H:%M:%S")
        self.schedule_time.setText(date_time)
    def schedule_week_button_setting(self):
        if self.schedule_day.isChecked():
            self.schedule_week_1.setEnabled(False)
            self.schedule_week_2.setEnabled(False)
            self.schedule_week_3.setEnabled(False)
            self.schedule_week_4.setEnabled(False)
            self.schedule_week_5.setEnabled(False)
            self.schedule_week_6.setEnabled(False)
            self.schedule_week_7.setEnabled(False)
        elif self.schedule_week.isChecked():
            self.schedule_week_1.setEnabled(True)
            self.schedule_week_2.setEnabled(True)
            self.schedule_week_3.setEnabled(True)
            self.schedule_week_4.setEnabled(True)
            self.schedule_week_5.setEnabled(True)
            self.schedule_week_6.setEnabled(True)
            self.schedule_week_7.setEnabled(True)
    def job_do(self, ini, idd):
        schedule_now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        self.printf(f"當前時間：{schedule_now}，開始執行爬蟲", 'schedule')
        path_name = setting_file_path + '/' + ini
        data,data_before,db_settings,db_settings_others = model.load_setting_ini(path_name)
        self.printf(f"{ini} 讀取完畢","schedule") 
        
        schedule_crawler = model.site_url_Spider()
        if data_before['before_url'] != '' and "選填" not in data_before['before_domain']:
            headers_attr = {"Host":data_before["before_domain"], "Content-Type":data_before["before_content_type"]}
            url_attr = {'former_part':data_before["before_url"], 'initial_pages':'', 'latter_part':'', 'end_pages':''}
            datas = eval(data_before["before_data"])
            method = data_before["before_method"]
            schedule_crawler.update_attr(idd, headers_attr, url_attr, '', '', '', datas, method)
            try:
                schedule_crawler.before_craw()
            except:
                self.printf('前置動作執行失敗，略過', 'schedule')
            self.printf('前置動作完成', 'schedule')

        if "url" in ini[0:3]:
            url_attr = {'former_part':data['former_part'], 'initial_pages':data['initial_pages'], 'latter_part':data['latter_part'], 'end_pages':data['end_pages']}
            headers_attr = {'Host':data['domain'],'Content-Type':None}
            schedule_crawler.update_attr(idd, headers_attr, url_attr, '', data['tag_section'], eval(data['tag_detail']), None, None)
            schedule_crawler.url_loop_parse()
        elif "tag" in ini[0:3]:
            url_attr = {'former_part':data['former_part'], 'initial_pages':data['initial_pages'], 'latter_part':data['latter_part'], 'end_pages':data['end_pages']}
            headers_attr = {'Host':data['domain'],'Content-Type':None}
            limit = int(data['upper_limit'])
            next_tag = eval(data['next_tag'])+tuple([limit])
            schedule_crawler.update_attr(idd, headers_attr, url_attr, next_tag, data['tag_section'], eval(data['tag_detail']), None, None)
            schedule_crawler.next_loop_parse()
        self.printf('爬取完成', 'schedule')
        self.printf(schedule_crawler.str, 'schedule')
        result = time.strftime("%Y%m%d_%H%M%S", time.localtime())
        if data['data_download'] == 'excel':
            self_file_path = data_file_path + '/' + idd + '_' + result + '.xlsx'
            schedule_crawler.export_excel(self_file_path)
            self.printf(f"檔案製作完成，檔案位置: {self_file_path}", 'schedule')
        elif data['data_download'] == 'json':
            self_file_path = data_file_path  + '/' + idd + '_' + result + '.txt'
            with open(self_file_path, 'a+', encoding='utf-8') as file:
                file.write(str(schedule_crawler.json))
                file.close()
            self.printf(f"檔案製作完成，檔案位置: {self_file_path}", 'schedule')
        elif data['data_download'] == 'sql':
            x=0
            dics = json.loads(schedule_crawler.json)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                for dic in dics:
                    if db_settings_others['is_datetime']=='sec':
                        dic['Data_Datetime']=datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M:%S")
                    elif db_settings_others['is_datetime']=='day':
                        dic['Data_Datetime']=datetime.datetime.strftime(datetime.date.today(), "%Y-%m-%d")
                    command_first = f"INSERT INTO {db_settings['db']}.{db_settings_others['table_name']} ({', '.join(dic.keys())}) VALUES "
                    command_second = '("'+ '\", \"'.join(dic.values()) +'") '
                    if db_settings_others['is_cover']:
                        update_query ='on DUPLICATE KEY UPDATE ' +  u', '.join([u"{}='{}'".format(k, v) for k, v in zip(dic.keys(),dic.values())])
                        command = command_first+command_second + update_query
                    else:
                        command = command_first+command_second
                    try:
                        cursor.execute(command)
                    except Exception as ex:
                        self.printf(f"{data} 存入資料庫報錯，錯誤內容: {ex}", 'schedule')
                        continue
                    x=x+1
            self.printf(f"資料庫儲存完成，資料共有 {len(dics)} 筆，成功存入 {x} 筆", 'schedule')
        self.printf(f"當前時間：{result}，執行爬蟲完成", 'schedule')
    def creat_job(self):
        self.schedule_create.setEnabled(False)
        if self.schedule_load_setting_comboBox.currentText().strip() == "":
            self.schedule_create.setEnabled(True)
            self.printf("未選擇設定檔", 'schedule')
            return
        if self.schedule_name.text().strip() == "":
            self.schedule_create.setEnabled(True)
            self.printf("排程名稱 不可為空！！", 'schedule')
            return
        if self.schedule_time.text().strip() == "" or "HH:MM:SS" in self.schedule_time.text().strip():
            self.schedule_create.setEnabled(True)
            self.printf("排程時間 不可為空！！", 'schedule')
            return
        if self.schedule_week.isChecked() and not (self.schedule_week_1.isChecked() or self.schedule_week_2.isChecked() or self.schedule_week_3.isChecked() or self.schedule_week_4.isChecked() or self.schedule_week_5.isChecked() or self.schedule_week_6.isChecked() or self.schedule_week_7.isChecked()):
            self.schedule_create.setEnabled(True)
            self.printf("請選擇每周幾執行", 'schedule')
            return
        
        dayOfWeek=''
        if self.schedule_day.isChecked():
            dayOfWeek='0-6,'
        elif self.schedule_week.isChecked():
            if self.schedule_week_1.isChecked():
                dayOfWeek=dayOfWeek+'0,'
            if self.schedule_week_2.isChecked():
                dayOfWeek=dayOfWeek+'1,'
            if self.schedule_week_3.isChecked():
                dayOfWeek=dayOfWeek+'2,'
            if self.schedule_week_4.isChecked():
                dayOfWeek=dayOfWeek+'3,'
            if self.schedule_week_5.isChecked():
                dayOfWeek=dayOfWeek+'4,'
            if self.schedule_week_6.isChecked():
                dayOfWeek=dayOfWeek+'5,'
            if self.schedule_week_7.isChecked():
                dayOfWeek=dayOfWeek+'6,'
        dayOfWeek=dayOfWeek[:-1]
        if "my_scheduler" not in str(dir(self)):
            self.my_scheduler = QtScheduler(timezone="Asia/Taipei")
            self.my_scheduler.start()
            self.my_scheduler_job_dic={}
            
        times = self.schedule_time.text().strip()
        times = times + ';'
        times = re.findall(r'(.*?);',times)
        schedule_name = self.schedule_name.text().strip()
        
        for time in times:
            x = 0
            try:
                time = datetime.datetime.strptime(time.strip(), "%H:%M:%S")
                hour=int(time.hour)
                minute=int(time.minute)
                second=int(time.second)
            except Exception as ex:
                self.printf("時間剖析錯誤，請檢查輸入", 'schedule')
                self.schedule_create.setEnabled(True)
                return
            
            self.Crontrigger = CronTrigger(timezone='Asia/Taipei', day_of_week=dayOfWeek, hour=hour, minute=minute, second=second)
            while True:
                try:
                    x = x + 1
                    idd = schedule_name+"_"+str(x)
                    self.my_scheduler.add_job(self.job_do, self.Crontrigger, id=idd, args=[self.schedule_load_setting_comboBox.currentText().strip(),idd] )
                    break
                except:
                    pass
            if x>1:
                duplicate_str = " (排程名稱重複，已重新修改名字)"
            else:
                duplicate_str = ""
            self.my_scheduler_job_dic[idd]=self.schedule_load_setting_comboBox.currentText().strip()
            self.printf(f"{str(hour)}:{str(minute)}:{str(second)} 名稱：{str(idd)} 排程建立成功。{duplicate_str}", 'schedule')

        self.schedule_create.setEnabled(True)
    def show_job(self):
        self.printf(' ', 'schedule')
        if "my_scheduler" in dir(self):
            jobs=self.my_scheduler.get_jobs()
            if len(self.my_scheduler_job_dic) ==0:
                self.printf('當前未存在任何排程', 'schedule')
                return
            for job in jobs:
                str = "名稱: %s, 觸發時間設定: %s, 下次執行時間: %s, 執行任務: %s" % (job.id, job.trigger, job.next_run_time, self.my_scheduler_job_dic[job.id])
                self.printf(str, 'schedule')
        else:
            self.printf('當前未新增任何一個任務', 'schedule')
            self.schedule_exec_all.setEnabled(True)
            self.schedule_delete.setEnabled(True)
    def delete_job(self):
        self.printf(' ', 'schedule')
        if len(self.my_scheduler_job_dic) > 0:
            for i in self.my_scheduler_job_dic:
                self.my_scheduler.remove_job(i,jobstore=None)
            self.my_scheduler_job_dic={}
            self.printf("刪除完畢", 'schedule')
        else:
            self.printf("當前無任務", 'schedule')
    def exec_job(self):
        self.printf(' ', 'schedule')
        if len(self.my_scheduler_job_dic) > 0:
            for ini in set(self.my_scheduler_job_dic.values()):
                idd = [k for k, v in self.my_scheduler_job_dic.items() if v == ini][0]
                self.job_do(ini, idd)
    def show_job_partial(self):
        self.printf(' ', 'schedule')
        if self.schedule_search.text().strip() =='':
            self.printf("請輸入排程名稱", 'schedule')
            return
        try:
            len(self.my_scheduler_job_dic)
        except:
            self.printf("當前未存在任何排程", 'schedule')
            return
        if len(self.my_scheduler_job_dic) > 0:
            jobs=self.my_scheduler.get_jobs()
            for job in jobs:
                if self.schedule_search.text().strip() in job.id :
                    str = "名稱: %s, 觸發時間設定: %s, 下次執行時間: %s, 執行任務: %s" % (job.id, job.trigger, job.next_run_time, self.my_scheduler_job_dic[job.id])
                    self.printf(str, 'schedule')
        else:
            self.printf("當前無任務", 'schedule')
        self.schedule_delete_search.setEnabled(True)
        self.schedule_exec_all_search.setEnabled(True)
    def delete_job_partial(self):
        self.printf(' ', 'schedule')
        if len(self.my_scheduler_job_dic) > 0:
            deleted_list=[]
            for job in self.my_scheduler_job_dic:
                if self.schedule_search.text().strip() in job:
                        self.my_scheduler.remove_job(job,jobstore=None)
                        deleted_list.append(job)
            [self.my_scheduler_job_dic.pop(key, None) for key in deleted_list]
            self.printf("刪除完畢", 'schedule')
        else:
            self.printf("當前無對應任務", 'schedule')
        self.schedule_delete_search.setEnabled(False)
        self.schedule_search.setText('')
    def exec_job_partial(self):
        self.printf(' ', 'schedule')
        if len(self.my_scheduler_job_dic) > 0:
            exec_list = {k:v for k, v in self.my_scheduler_job_dic.items() if self.schedule_search.text().strip() in k}
            for ini in set(exec_list.values()):
                idd = [k for k, v in exec_list.items() if v == ini][0]
                self.job_do(ini, idd)

In [ ]:
from builtins import str
if __name__ == "__main__":
    ui = controller()
    ui.MainWindow.show()
    try:
        sys.exit(ui.app.exec_())
    except:
        pass
    del ui.app
    del ui.MainWindow
    del ui